In [1]:
import torch
import os, sys
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.9.1+cu102 (GeForce RTX 2060)


In [2]:
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
mult = 1024 * 1024
print(t / mult, r / mult, a / mult, f / mult)

5934.5625 0.0 0.0 0.0


In [3]:
%%markdown
## Dataset labels remapping for COCO 80 classes

## Dataset labels remapping for COCO 80 classes


In [4]:
old_names = ['bird', 'cat', 'dog', 'person']
new_names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
        'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
        'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
        'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
        'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
        'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
        'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
        'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
        'hair drier', 'toothbrush']

In [5]:
remap_dict = dict([(str(i), str(new_names.index(label))) for i, label in enumerate(old_names)])
remap_dict

{'0': '14', '1': '15', '2': '16', '3': '0'}

In [21]:
dataset_labels_location = '../datasets/hack_coco_finetune/valid/labels/'

In [22]:
label_files = os.listdir(dataset_labels_location)
label_files = [os.path.join(dataset_labels_location, lf) for lf in label_files]
label_files[:5]

['../datasets/hack_coco_finetune/valid/labels/6255_734_jpg.rf.1253ef88fb7007039bfc68bc41126ed7.txt',
 '../datasets/hack_coco_finetune/valid/labels/8569_2492_jpg.rf.11a478189ac0241e10a2af35aadaa8f3.txt',
 '../datasets/hack_coco_finetune/valid/labels/6641_2361_jpg.rf.7450323e5511fa6be93cda47d38d26c3.txt',
 '../datasets/hack_coco_finetune/valid/labels/7831_183_png.rf.b2b9f0a64fb6eda7f81fa4b2ee1a99b2.txt',
 '../datasets/hack_coco_finetune/valid/labels/7650_2207_jpg.rf.090a714164eeb561f69f8856f680d0aa.txt']

In [24]:
# for lf in label_files:
#     fixed_lines = []
#     with open(lf, 'r', encoding="utf-8") as file:
#         all_lines = file.readlines()
#         for line in all_lines:
#             splitted = line.split(' ')
#             splitted[0] = remap_dict[splitted[0]]
#             fixed_lines.append(" ".join(splitted))
#     with open(lf, 'w', encoding="utf-8") as file:
#         file.writelines(fixed_lines)

In [28]:
# for lf in label_files:
#     fixed_lines = []
#     with open(lf, 'r', encoding="utf-8") as file:
#         all_lines = file.readlines()
#         for line in all_lines:
#             splitted = line.split(' ')
#             if splitted[0] == '14':
#                 continue
#             if splitted[0] == '15':
#                 splitted[0] = '0'
#             if splitted[0] == '16':
#                 splitted[0] = '1'
#             if splitted[0] == '0':
#                 splitted[0] = '2'
#             fixed_lines.append(" ".join(splitted))
#     with open(lf, 'w', encoding="utf-8") as file:
#         file.writelines(fixed_lines)

In [26]:
# for lf in label_files:
#     with open(lf, 'r', encoding="utf-8") as file:
#         print(lf, '\n', file.readlines())

In [19]:
!cd ../yolov5/

/home/hivaze/CODE-W/PyCharm/dogfound/neural_network/yolov5


In [27]:
!pwd

/home/hivaze/CODE-W/PyCharm/dogfound/neural_network/yolov5


In [14]:
%%markdown
## Train stage

## Train stage


In [15]:
# %load_ext tensorboard
# %tensorboard --logdir runs/train

In [16]:
%%markdown
### Проверка базовой модели yolov5l на датасете

### Проверка базовой модели yolov5l на датасете


In [31]:
!python val.py --weights yolov5l.pt --verbose --task train --data hack_data.yaml --img 640 --batch-size 32 --conf-thres 0.05 --iou-thres 0.45 --half

val: data=/home/hivaze/CODE-W/PyCharm/dogfound/neural_network/yolov5/data/hack_data.yaml, weights=['yolov5l.pt'], batch_size=32, imgsz=640, conf_thres=0.05, iou_thres=0.45, task=train, device=, single_cls=False, augment=False, verbose=True, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=True
YOLOv5 🚀 v6.0-62-g5f603a9 torch 1.9.1+cu102 CUDA:0 (GeForce RTX 2060, 5935MiB)

Fusing layers... 
Model Summary: 367 layers, 46533693 parameters, 0 gradients
train: Scanning '../datasets/hack_coco_finetune/train/labels' images and labels.
train: New cache created: ../datasets/hack_coco_finetune/train/labels.cache
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        895       1308      0.717      0.472      0.613       0.43
              person        895        390      0.828      0.797      0.837      0.525
                bird        895         65      0.408      0.508  

In [32]:
%%markdown
### Файн-тьюнинг на датасете

### Файн-тьюнинг на датасете


In [48]:
!python train.py --img 600 --batch 8 --hyp hyp.finetune.yaml --epochs 11 --data hack_data.yaml --weights yolov5l.pt --project "projects/COCO-street-finetune-2" --cache

train: weights=yolov5l.pt, cfg=, data=hack_data.yaml, hyp=hyp.finetune.yaml, epochs=11, batch_size=8, imgsz=600, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=projects/COCO-street-finetune-2, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 4 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.0-62-g5f603a9 torch 1.9.1+cu102 CUDA:0 (GeForce RTX 2060, 5935MiB)

hyperparameters: lr0=0.0032, lrf=0.12, momentum=0.843, weight_decay=0.00036, warmup_epochs=2.0, warmup_momentum=0.5, warmup_bias_lr=0.05, box=0.0296, cls=0.243, cls_pw=0.631, obj=0.301, obj_pw=0.911, iou_t=0.2, anchor_t=2.9


     Epoch   gpu_mem       box       obj       cls    labels  img_size
      9/10     4.73G   0.01994  0.002911   0.00578        13       608: 100%|█| 
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        164        312      0.832      0.869      0.902      0.522

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     10/10     4.73G   0.01976  0.002756  0.005103        38       608: 100%|█| 
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        164        312      0.863       0.84      0.903      0.535

11 epochs completed in 0.160 hours.
Optimizer stripped from projects/COCO-street-finetune-2/exp/weights/last.pt, 93.6MB
Optimizer stripped from projects/COCO-street-finetune-2/exp/weights/best.pt, 93.6MB

Validating projects/COCO-street-finetune-2/exp/weights/best.pt...
Fusing layers... 
Model Summary: 367 layers, 46533693 parameters, 0 gradients, 109.1 G

In [38]:
%%markdown
### Проверка файнтьюненной модели на трейне

### Проверка файнтьюненной модели на трейне


In [49]:
!python val.py --weights projects/COCO-street-finetune-2/exp/weights/best.pt --verbose --task train --data hack_data.yaml --img 640 --batch-size 32 --conf-thres 0.05 --iou-thres 0.45 --half

val: data=/home/hivaze/CODE-W/PyCharm/dogfound/neural_network/yolov5/data/hack_data.yaml, weights=['projects/COCO-street-finetune-2/exp/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.05, iou_thres=0.45, task=train, device=, single_cls=False, augment=False, verbose=True, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=True
YOLOv5 🚀 v6.0-62-g5f603a9 torch 1.9.1+cu102 CUDA:0 (GeForce RTX 2060, 5935MiB)

Fusing layers... 
Model Summary: 367 layers, 46533693 parameters, 0 gradients, 109.1 GFLOPs
train: Scanning '../datasets/hack_coco_finetune/train/labels.cache' images and l
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        895       1308      0.892      0.762      0.865      0.602
              person        895        390      0.899      0.794      0.884       0.58
                bird        895         65       0.93      0.677      0.808      0.472


In [34]:
%%markdown
### Полноценное обучение с нуля

### Полноценное обучение с нуля


In [43]:
!python train.py --img 600 --batch 8 --epochs 25 --data hack_data_scratch.yaml --weights yolov5l.pt --project "projects/COCO-street-from-scratch-3" --cache

train: weights=yolov5l.pt, cfg=, data=hack_data_scratch.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=21, batch_size=8, imgsz=600, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=projects/COCO-street-from-scratch-3, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 4 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.0-62-g5f603a9 torch 1.9.1+cu102 CUDA:0 (GeForce RTX 2060, 5935MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor


     Epoch   gpu_mem       box       obj       cls    labels  img_size
      9/20     4.65G   0.05062   0.01158   0.01139        23       608: 100%|█| 
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        164        312      0.677      0.667      0.698      0.281

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     10/20     4.65G   0.04757   0.01125   0.01012        21       608: 100%|█| 
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        164        312      0.762      0.698      0.768      0.351

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     11/20     4.65G   0.04432   0.01099  0.008552        22       608: 100%|█| 
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        164        312      0.823      0.775      0.849      0.419

     Epoch   gpu_mem       box      

In [44]:
%%markdown
### Проверка обученной с нуля модели на трейне

### Проверка обученной с нуля модели на трейне


In [50]:
!python val.py --weights "projects/COCO-street-from-scratch-3/exp/weights/best.pt" --verbose --task train --data hack_data_scratch.yaml --img 640 --batch-size 32 --conf-thres 0.05 --iou-thres 0.45 --half

val: data=/home/hivaze/CODE-W/PyCharm/dogfound/neural_network/yolov5/data/hack_data_scratch.yaml, weights=['projects/COCO-street-from-scratch-3/exp/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.05, iou_thres=0.45, task=train, device=, single_cls=False, augment=False, verbose=True, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=True
YOLOv5 🚀 v6.0-62-g5f603a9 torch 1.9.1+cu102 CUDA:0 (GeForce RTX 2060, 5935MiB)

Fusing layers... 
Model Summary: 367 layers, 46124433 parameters, 0 gradients, 107.8 GFLOPs
train: Scanning '../datasets/hack_scratch/train/labels.cache' images and labels.
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        895       1308      0.924      0.957      0.972       0.68
                bird        895         65      0.875      0.967      0.963      0.568
                 cat        895        166          1      0.969      0.989

In [35]:
%%markdown
### Проверки на COCO val датасете

### Проверки на COCO val датасете


In [36]:
# Run YOLOv5x on COCO val2017
!python val.py --weights yolov5l.pt --verbose --data coco.yaml --img 640 --batch-size 32 --conf-thres 0.05 --iou-thres 0.45 --half

val: data=/home/hivaze/CODE-W/PyCharm/dogfound/neural_network/yolov5/data/coco.yaml, weights=['yolov5l.pt'], batch_size=32, imgsz=640, conf_thres=0.05, iou_thres=0.45, task=val, device=, single_cls=False, augment=False, verbose=True, save_txt=False, save_hybrid=False, save_conf=False, save_json=True, project=runs/val, name=exp, exist_ok=False, half=True
YOLOv5 🚀 v6.0-62-g5f603a9 torch 1.9.1+cu102 CUDA:0 (GeForce RTX 2060, 5935MiB)

Fusing layers... 
Model Summary: 367 layers, 46533693 parameters, 0 gradients
val: Scanning '../datasets/coco/val2017' images and labels...4952 found, 48 miss
val: New cache created: ../datasets/coco/val2017.cache
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all       5000      36335      0.754      0.605      0.683      0.508
              person       5000      10777      0.841      0.743      0.837      0.623
             bicycle       5000        314      0.781      0.567      0.674      0.426
        

DONE (t=18.68s).
Accumulating evaluation results...
DONE (t=2.56s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.471
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.654
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.511
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.303
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.524
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.603
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.360
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.556
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.575
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.386
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.628
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= la

In [37]:
!python val.py --weights projects/COCO-street-finetune/exp/weights/best.pt --verbose --data coco.yaml --img 640 --batch-size 32 --conf-thres 0.05 --iou-thres 0.45 --half

val: data=/home/hivaze/CODE-W/PyCharm/dogfound/neural_network/yolov5/data/coco.yaml, weights=['projects/COCO-street-finetune/exp/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.05, iou_thres=0.45, task=val, device=, single_cls=False, augment=False, verbose=True, save_txt=False, save_hybrid=False, save_conf=False, save_json=True, project=runs/val, name=exp, exist_ok=False, half=True
YOLOv5 🚀 v6.0-62-g5f603a9 torch 1.9.1+cu102 CUDA:0 (GeForce RTX 2060, 5935MiB)

Fusing layers... 
Model Summary: 367 layers, 46533693 parameters, 0 gradients, 109.1 GFLOPs
val: Scanning '../datasets/coco/val2017.cache' images and labels... 4952 found, 
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all       5000      36335      0.678       0.55      0.635      0.464
              person       5000      10777      0.746      0.765      0.819      0.607
             bicycle       5000        314      0.776      0.436      0.611      0.403
          

DONE (t=13.89s).
Accumulating evaluation results...
DONE (t=2.05s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.366
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.526
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.406
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.202
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.414
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.512
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.303
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.437
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.445
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.238
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.484
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= la

In [17]:
# Run YOLOv5x on COCO val2017
!python val.py --weights projects/COCO-street-finetune-6/exp/weights/best.pt --verbose --data coco.yaml --img 640 --batch-size 32 --conf-thres 0.1 --iou-thres 0.45 --half

val: data=/home/hivaze/CODE-W/PyCharm/lct_hack/yolov5/data/coco.yaml, weights=['projects/COCO-street-finetune-6/exp/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.1, iou_thres=0.45, task=val, device=, single_cls=False, augment=False, verbose=True, save_txt=False, save_hybrid=False, save_conf=False, save_json=True, project=runs/val, name=exp, exist_ok=False, half=True
YOLOv5 🚀 v6.0-13-gfc36064 torch 1.9.1+cu102 CUDA:0 (GeForce RTX 2060, 5934.5625MB)

Fusing layers... 
Model Summary: 367 layers, 46533693 parameters, 0 gradients, 109.1 GFLOPs
val: Scanning '../datasets/coco/val2017.cache' images and labels... 4952 found, 
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all       5000      36335      0.684      0.553      0.642      0.477
              person       5000      10777      0.753      0.761      0.815      0.609
             bicycle       5000        314      0.754      0.459      0.634      0.434
                 car

DONE (t=11.12s).
Accumulating evaluation results...
DONE (t=1.71s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.355
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.503
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.391
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.184
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.400
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.521
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.295
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.416
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.420
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.210
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.456
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= la

In [ ]:
# Run YOLOv5x on COCO val2017
!python val.py --weights projects/COCO-street-finetune-6/exp/weights/best.pt --verbose --data coco.yaml --img 640 --batch-size 32 --conf-thres 0.1 --iou-thres 0.45 --half